### Numerical weather prediction (NWP) example
Code to replicate the CPA plots to compare the performance of the HRES forecast and the persistence forecast for the variables 2m temperature, wind speed and precipitation for different leadtimes. As respective observation the ERA5 reanalysis product is taken. The data can be downloaded from the following sources:
- https://confluence.ecmwf.int/display/TIGGE
- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

Before the netCDF files were imported into Python they were modified by CDO.

The following code computes the CPA plot for precipitation. To obtain the respective plots for wind speed and Temperature replace the name of the variable.

In [ ]:
# Import packages
import xarray as xr 
import numpy as np
from netCDF4 import Dataset
from urocc import cpa
import matplotlib.pyplot as plt

#### CPA between Persistence and Reanalysis 
The persistence forecast uses the present condition as forecast for future conditions. For this comparison the ERA5 reanalysis product is used.

Computation performed for precipitation. For 2m temperature replace variable name *tp* in function *cpa_prep* to *t2m*.

In [ ]:
def cpa_prep(response, predictor):
    # varible name: tp for precipitation
    #               t2m for 2m temperature
    response2 = response.tp.values.flatten()
    predictor2 = predictor.tp.values.flatten()
    return cpa(response2, predictor2)

In [ ]:
target_obs = 'Precip_Reanalysis_2006-2018.nc'
DS_observation = xr.open_dataset(target_obs)

# set lead time: For Leadtime of one day ahead use lead=1 and for Leadtime of 5 days ahead use lead=5
lead = 1
# initialize
cpa_list = []

for i in range(11):
    year = 2007 + i
    year2 = 2008 + i
    tsel1 = pd.date_range(start='01/01/'+str(year), end='12/31/'+str(year))
    tsel2 = pd.date_range(start='04/01/'+str(year), end='03/31/'+ str(year2))
    tsel3 = pd.date_range(start='07/01/'+str(year), end='06/30/'+ str(year2))
    tsel4 = pd.date_range(start='10/01/'+str(year), end='09/30/'+ str(year2))
    tsel5 = tsel1 -  pd.to_timedelta(lead, unit='d')
    tsel6 = tsel2 -  pd.to_timedelta(lead, unit='d')
    tsel7 = tsel3 -  pd.to_timedelta(lead, unit='d')
    tsel8 = tsel4 -  pd.to_timedelta(lead, unit='d')
    response1 = DS_observation.sel(time = tsel1)
    response2 = DS_observation.sel(time = tsel2)
    response3 = DS_observation.sel(time = tsel3)
    response4 = DS_observation.sel(time = tsel4)
    
    predictor1 = DS_observation.sel(time = tsel5)
    predictor2 = DS_observation.sel(time = tsel6)
    predictor3 = DS_observation.sel(time = tsel7)
    predictor4 = DS_observation.sel(time = tsel8)
    
    cpa_list.append(cpa_prep(response1, predictor1))
    cpa_list.append(cpa_prep(response2, predictor2))
    cpa_list.append(cpa_prep(response3, predictor3))
    cpa_list.append(cpa_prep(response4, predictor4))

tsel1 = pd.date_range(start='01/01/2018', end='12/31/2018')
tsel5 = tsel1 -  pd.to_timedelta(lead, unit='d')
response1 = DS_observation.sel(time = tsel1)
predictor1 = DS_observation.sel(time = tsel5)
cpa_list.append(cpa(response1, predictor1))

np.savetxt('CPA_Precipitation_per_24.txt',cpa_list)

#### CPA between HRES and Reanalysis 

Hres from tigge is used as forecast and compared to the ERA5 reanalysis data.

In [ ]:
target_obs = 'Precipitation_Reanalysis_2006-2018.nc'
DS_observation = xr.open_dataset(target_obs)
target_hres = 'Precipitation_hres_24.nc'
DS_predictor = xr.open_dataset(target_hres)

lead = 1
cpa_list = []
# indices for reanalysis and hres
for i in range(11):
    year = 2007 + i
    year2 = 2008 + i
    tsel1 = pd.date_range(start='01/01/'+str(year), end='12/31/'+str(year))
    tsel2 = pd.date_range(start='04/01/'+str(year), end='03/31/'+ str(year2))
    tsel3 = pd.date_range(start='07/01/'+str(year), end='06/30/'+ str(year2))
    tsel4 = pd.date_range(start='10/01/'+str(year), end='09/30/'+ str(year2))
    
    response1 = DS_observation.sel(time = tsel1)
    response2 = DS_observation.sel(time = tsel2)
    response3 = DS_observation.sel(time = tsel3)
    response4 = DS_observation.sel(time = tsel4)
    
    predictor1 = DS_predictor.sel(time = tsel1)
    predictor2 = DS_predictor.sel(time = tsel2)
    predictor3 = DS_predictor.sel(time = tsel3)
    predictor4 = DS_predictor.sel(time = tsel4)
    
    cpa_list.append(cpa_prep(response1, predictor1))
    cpa_list.append(cpa_prep(response2, predictor2))
    cpa_list.append(cpa_prep(response3, predictor3))
    cpa_list.append(cpa_prep(response4, predictor4))

tsel1 = pd.date_range(start='01/01/2018', end='12/31/2018')
response1 = DS_observation.sel(time = tsel1)
predictor1 = DS_predictor.sel(time = tsel1)
cpa_list.append(cpa_prep(response1, predictor1))

np.savetxt('CPA_Precipitation_hres_24.txt',cpa_list)

## Plot

Plot of CPA for Persistence and HRES for different leadtimes over time

In [ ]:
lead1_per = np.loadtxt('CPA_Precipitation_pre_24.txt')
lead2_per = np.loadtxt('CPA_Precipitation_pre_48.txt')
lead3_per = np.loadtxt('CPA_Precipitation_pre_72.txt')
lead4_per = np.loadtxt('CPA_Precipitation_pre_96.txt')
lead5_per = np.loadtxt('CPA_Precipitation_pre_120.txt')

lead1_hres = np.loadtxt('CPA_Precipitation_hres_24.txt')
lead2_hres = np.loadtxt('CPA_Precipitation_hres_48.txt')
lead3_hres = np.loadtxt('CPA_Precipitation_hres_72.txt')
lead4_hres = np.loadtxt('CPA_Precipitation_hres_96.txt')
lead5_hres = np.loadtxt('CPA_Precipitation_hres_120.txt')

In [ ]:
color_blue = np.array(["#9ECAE1", "#6BAED6", "#4292C6", "#2171B5", "#084594"])
color_green = np.array(["#A1D99B", "#74C476", "#41AB5D", "#238B45", "#005A32"])

plt.plot(np.arange(45), lead1_per[0:45], '.', color = color_green[4], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead1_per[np.arange(0, 45, 4)], color = color_green[4])
plt.plot(np.arange(45), lead2_per[0:45], '.', color = color_green[3], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead2_per[np.arange(0, 45, 4)], color = color_green[3])
plt.plot(np.arange(45), lead3_per[0:45], '.', color = color_green[2], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead3_per[np.arange(0, 45, 4)], color = color_green[2])
plt.plot(np.arange(45), lead4_per[0:45], '.', color = color_green[1], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead4_per[np.arange(0, 45, 4)], color = color_green[1])
plt.plot(np.arange(45), lead5_per[0:45], '.', color = color_green[0], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead5_per[np.arange(0, 45, 4)], color = color_green[0])

plt.plot(np.arange(45), lead1_hres, '.',color = color_blue[4], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead1_hres[np.arange(0, 45, 4)],color = color_blue[4])
plt.plot(np.arange(45), lead2_hres, '.',color = color_blue[3], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead2_hres[np.arange(0, 45, 4)],color = color_blue[3])
plt.plot(np.arange(45), lead3_hres, '.',color = color_blue[2], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead3_hres[np.arange(0, 45, 4)],color = color_blue[2])
plt.plot(np.arange(45), lead4_hres, '.',color = color_blue[1], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead4_hres[np.arange(0, 45, 4)],color = color_blue[1])
plt.plot(np.arange(45), lead5_hres, '.',color = color_blue[0], markersize = 1.5)
plt.plot(np.arange(0, 45, 4), lead5_hres[np.arange(0, 45, 4)],color = color_blue[0])

plt.xlabel('Year')
plt.ylabel('CPA')
ticks = plt.xticks(np.arange(4,45,12),[2008, 2011, 2014, 2017])  